## Clifford Attractor

A [Clifford Attractor](http://paulbourke.net/fractals/clifford) is defined by a set of iterative equations that determine discrete steps in the path of a particle across a 2D space, given a starting point _(x0,y0)_ and the values of four parameters _(a,b,c,d)_:

\begin{equation}
x_{n +1} = \sin(a y_{n}) + c \cos(a x_{n})\\
y_{n +1} = \sin(b x_{n}) + d \cos(b y_{n})
\end{equation}

At each time step, the equations define the location for the following time step, and the accumulated locations show the areas of the 2D plane most commonly visited by the particle.  

It's easy to calculate these values in Python using [Numba](http://numba.pydata.org) and to visualize them with [Datashader](http://datashader.org), using code adapted from [L&aacute;zaro Alonso](https://lazarusa.github.io/Webpage/codepython2.html). 

First, we define the iterative attractor equation:

In [ ]:
import numpy as np, pandas as pd, datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import inferno, viridis
from numba import jit

@jit
def clifford(a, b, c, d, x, y):
    return np.sin(a * y) + c * np.cos(a * x), \
           np.sin(b * x) + d * np.cos(b * y)

We then evaluate this equation many times, creating a set of (x,y) coordinates visited. The @jit here and above is optional, but it makes the code 50x faster.

In [ ]:
n=10000000

@jit
def trajectory(fn, a, b, c, d, x0=0, y0=0, n=n):
    x, y = np.zeros(n), np.zeros(n)
    x[0], y[0] = x0, y0
    for i in np.arange(n-1):
        x[i+1], y[i+1] = fn(a, b, c, d, x[i], y[i])
    return pd.DataFrame(dict(x=x,y=y))

In [ ]:
%%time
df = trajectory(clifford, -1.3, -1.3, -1.8, -1.9, 0, 0)

In [ ]:
df.tail()

We can now aggregate these coordinates into a 2D grid:

In [ ]:
%%time

cvs = ds.Canvas(plot_width = 400, plot_height = 400)
agg = cvs.points(df, 'x', 'y')

So that we can see the grid, we need to turn the integer count-per-cell values into colors.  There are infinitely many ways to do that, of which let's look at four:

In [ ]:
wb  = ["white", "black"]
wb2 = ["#ffffff","#f0f0f0","#d9d9d9","#bdbdbd","#969696","#737373","#525252","#252525","#000000"]

ds.transfer_functions.Image.border=0

tf.Images(tf.shade(agg, cmap = inferno[::-1]),
          tf.shade(agg, cmap = viridis[::-1]),
          tf.shade(agg, cmap = wb),
          tf.shade(agg, cmap = wb2, how='linear', span = [0,n/100000])).cols(2)

Here the first three use Datashader's default histogram equalization, while the last uses a custom linear cropped colormap that highlights the most-visited regions. Let's collect some other suitable [white-background, uniform sequential colormaps](http://holoviews.org/user_guide/Colormaps.html) for subsequent plots:

In [ ]:
from colorcet import palette
cmaps =  [palette[p][::-1] for p in ['bgy', 'bmw', 'bgyw', 'bmy','fire', 'gray', 'kbc', 'kgy']]
cmaps += [inferno[::-1], viridis[::-1]]

You can get a variety of trajectories if you use different parameter values, and a variety of different appearances depending on colormap:

In [ ]:
def plot(fn, a, b, c, d, x0=0, y0=0, n=n, cmap=wb, label=False):
    label = "{}, {}, {}, {}".format(a,b,c,d) if label else None
    df  = trajectory(fn, a, b, c, d, x0, y0, n)
    cvs = ds.Canvas(plot_width = 400, plot_height = 400)
    agg = cvs.points(df, 'x', 'y')
    img = tf.shade(agg, cmap=cmap, name=label)
    return img

cvals = [
    (-1.3,   -1.3,   -1.8,   -1.9),
    (-1.4,    1.6,    1.0,    0.7),
    ( 1.7,    1.7,    0.6,    1.2),
    ( 1.5,   -1.8,    1.6,    0.9),
    (-1.7,    1.8,   -1.9,   -0.4),
    ( 1.1,   -1.32,  -1.03,   1.54),
    (-1.9,   -1.9,   -1.9,   -1.0),
    ( 0.77,   1.99,  -1.31,  -1.45),
    (0.75,    1.34,  -1.93,   1.00),
    (-1.32,  -1.65,   0.74,   1.81),
]

tf.Images(*[plot(clifford, *cvals[i], cmap=cmaps[i%len(cmaps)]) for i in range(len(cvals))]).cols(5)

The above examples are selected for illustration, but randomly sampling the parameter space will show that there are less interesting parameter combinations as well, such as all values being on a single fixed point:

In [ ]:
import numpy.random
np.random.seed(12)
rvals=np.random.random((5,4))*4-2

tf.Images(*[plot(clifford, *rvals[i], cmap=cmaps[(i+1)%len(cmaps)], label=True) 
            for i in range(len(rvals))]).cols(5)

If you wish, datashader could easily be used to filter out such uninteresting examples, by applying a criterion to the aggregate array before shading and showing only those that remain (e.g. rejecting those where 80% of the pixel bins are empty).


## De Jong attractors

A related set of attractors was proposed by [Peter de Jong](http://paulbourke.net/fractals/peterdejong):

In [ ]:
@jit
def dejong(a, b, c, d, x, y):
    return np.sin(a * y) - np.cos(b * x), \
           np.sin(c * x) - np.cos(d * y)

dvals = [
    (-1.244, -1.251, -1.815, -1.908),
    ( 1.7,    1.7,    0.6,    1.2),
    ( 1.4,   -2.3,    2.4,   -2.1),
    (-2.7,   -0.09,  -0.86,  -2.2),
    (-0.827, -1.637,  1.659, -0.943),
    (-2.24,   0.43,  -0.65,  -2.43),
    (-2,     -2,     -1.2,    2.0),
    (-0.709,  1.638,  0.452,  1.740),
    ( 2.01,  -2.53,   1.61,  -0.33),
    ( 1.40,   1.56,   1.40,  -6.56)]

tf.Images(*[plot(dejong, *dvals[i], cmap=cmaps[1-i]) for i in range(len(dvals))]).cols(5)

## Svensson attractors

Another variation was provided by Johnny Svensson:

In [ ]:
@jit
def svensson(a, b, c, d, x, y):
    return d * np.sin(a * x) - np.sin(b * y), \
           c * np.cos(a * x) + np.cos(b * y)

svals = [
    ( 1.5,   -1.8,    1.6,    0.9),
    (-0.91,  -1.251, -1.815, -1.908),
    (-1.78,   1.29,  -0.09,  -1.18),
    (-0.91,  -1.29,  -1.97,  -1.56),
    ( 1.40,   1.56,   1.40,  -6.56)]

tf.Images(*[plot(svensson, *svals[i], cmap=cmaps[i]) for i in range(len(svals))]).cols(5)

## Interactive plotting

If you are running a live Python process, you can use Datashader with HoloViews and Bokeh to zoom in and see the individual steps in these calculations:

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade, dynspread
hv.extension('bokeh')

dynspread(datashade(hv.Points(trajectory(clifford, *cvals[5])), cmap=viridis[::-1]).options(width=400,height=400))

Each time you zoom in in a live process, the data will be reaggregated, which should take a small fraction of a second for 10 million points.  Eventually, once you zoom in enough you should see individual data points, as we are not connecting the points into a trajectory here. 

You can also try "connecting the dots", which will reveal how the particle jumps from one region of the space to another:

In [ ]:
dynspread(datashade(hv.Path([trajectory(clifford, *cvals[5])]), cmap=viridis[::-1]).options(width=400,height=400))

Again, if you zoom in on a live server, the plot will update so that you can see the individual traces involved. 

On the live server, you can also explore to find your own parameter values that generate interesting patterns:

In [ ]:
def hv_clif(a,b,c,d,x0=0,y0=0,n=n):
    return datashade(hv.Points(trajectory(clifford, a, b, c, d, x0, y0, n)), cmap=inferno[::-1], dynamic=False)

a,b,c,d=cvals[3]

dm = hv.DynamicMap(hv_clif, kdims=['a', 'b', 'c', 'd'])
dm = dm.redim.range(a=(-2.0, 2.0), b=(-2.0,2.0), c=(-2.0,2.0), d=(-2.0,2.0))
dm = dm.redim.default(a=a, b=b, c=c, d=d).options(width=500,height=500)
dm

Although many of the regions of this four-dimensional parameter space generate uninteresting trajectories such as single points, you can find interesting regions by starting with one of the a,b,c,d tuples of values in previous plots, then click on one slider and use the left and right arrow keys to see how the plot changes as that parameter changes.